The goal of this workbook is to end with our city dataset which I will use for analysis. It will include 4 different sources of data.

City Dataset
- 2013 city GHG emissions per capita
- 2017 Mayoral support (signature) for Paris climate agreement
- 2000-2011 City Partisan index
- Mayoral political party (for a mayor that had the opportunity to sign support for paris)

Import Modules

In [35]:
from os import path
import requests
from bs4 import BeautifulSoup as Soup
from urllib.request import Request, urlopen
from pandas import DataFrame
import pandas as pd
import numpy as np
import math
DATA_DIR = '/Users/joeskvarna/Desktop/Thesis/Data'
#pd.set_option('display.max_rows', None)
#pd.set_option('max_rows', 5)

# 1. GHG emissions Per Capita

Open txt file from source and take a look

In [36]:
city = pd.read_csv(path.join(DATA_DIR, 'city_ghg_source.txt'), sep =  '\t')
city.head()

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking
0,Hong Kong SAR,China,6029000,NaN,37.8,34.6 &plusmn;6.3,13847,1
1,Mohammed Bin Zayed City,UAE,188000,NaN,5.3,32.9 &plusmn;27.9,5292,2
2,Abu Dhabi,UAE,911000,NaN,15.6,32.9 &plusmn;17.1,5288,3
3,Country of Singapore,Singapore,5235000,NaN,34.1,30.8 &plusmn;6.5,13845,4
4,Hulun Buir,China,198000,NaN,6.4,30.0 &plusmn;32.3,10441,5


select only cities from the US

In [37]:
city_us = city.loc[city['Country'] == 'USA']
city_us = city_us.reset_index(drop=True)
city_us.sample(10)

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking
75,Miami,USA,5031000,NaN,21.9,16.0 &plusmn;4.4,513,119
62,Salem,USA,150000,NaN,1.7,16.8 &plusmn;11.4,95,97
108,"Unknown city at lat/lon 27.3, -80.4",USA,170000,NaN,0.8,9.2 &plusmn;5.0,550,471
52,San Jose,USA,4003000,NaN,19.7,17.5 &plusmn;4.9,11,85
63,Portland,USA,1163000,NaN,9.3,16.7 &plusmn;8.0,104,102
66,Columbia,USA,177000,NaN,1.6,16.6 &plusmn;8.9,637,106
65,Allentown,USA,359000,NaN,3.4,16.6 &plusmn;9.5,837,104
3,St. Louis,USA,1177000,NaN,9.6,24.1 &plusmn;8.2,448,19
32,Buffalo,USA,570000,NaN,5.7,19.8 &plusmn;10.0,16274,56
82,Palm Springs,USA,299000,NaN,2.3,15.4 &plusmn;7.5,61,131


Some cities specificy their state with ",MA" for example. We want to remove this to keep name format consistent

In [38]:
city_us["Urban Cluster"] = city_us["Urban Cluster"].str.replace(r',[^,]*$', '')
city_us.sample(10)

<ipython-input-38-45e02f0c8557>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  city_us["Urban Cluster"] = city_us["Urban Cluster"].str.replace(r',[^,]*$', '')


,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking
53,Rochester,USA,306000,NaN,4.2,17.5 &plusmn;13.9,836,86
91,El Paso,USA,463000,NaN,4.0,14.6 &plusmn;8.5,14661,159
37,Hartford,USA,568000,NaN,5.9,19.0 &plusmn;10.4,904,63
67,Corpus Christi,USA,179000,NaN,1.8,16.6 &plusmn;10.2,254,107
6,Kansas City,USA,1004000,NaN,7.4,22.6 &plusmn;7.4,386,23
25,Naples,USA,200000,NaN,2.5,20.5 &plusmn;12.4,471,46
26,Milwaukee,USA,821000,NaN,7.3,20.4 &plusmn;8.9,628,47
90,Los Angeles,USA,13482000,NaN,43.7,14.6 &plusmn;3.2,15,158
104,Unknown city at lat/lon 40.3,USA,253000,NaN,2.1,11.0 &plusmn;8.2,193,330
17,Burnsville,USA,152000,NaN,2.0,21.2 &plusmn;12.9,521,37


This is good fo now, we will bring in our other data

# 2. Mayoral support for Paris Climate Agreement

This data is taken from a website. We will scrape the data from this site.

In [39]:
url = 'https://climatemayors.org/actions-paris-climate-agreement'
def scraper(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    page_soup = Soup(webpage, 'html.parser')
    return page_soup
page_soup = scraper(url)

Look at a list of all the paragraph tags on the website

In [40]:
p = page_soup.find_all('p')
print(p[:10])
print(p[-10:])

[<p>STATEMENT FROM THE CLIMATE MAYORS IN RESPONSE TO PRESIDENT TRUMP’S WITHDRAWAL FROM THE PARIS CLIMATE AGREEMENT</p>, <p>The President’s denial of global warming is getting a cold reception from America’s cities.</p>, <p>As 466 US Mayors representing 74 million Americans, we will adopt, honor, and uphold the commitments to the goals enshrined in the Paris Agreement. We will intensify efforts to meet each of our cities’ current climate goals, push for new action to meet the 1.5 degrees Celsius target, and work together to create a 21st century clean energy economy.</p>, <p>We will continue to lead. We are increasing investments in renewable energy and energy efficiency. We will buy and create more demand for electric cars and trucks. We will increase our efforts to cut greenhouse gas emissions, create a clean energy economy, and stand for environmental justice. And if the President wants to break the promises made to our allies enshrined in the historic Paris Agreement, we’ll build an

For each p tag with city and mayor, split the lines to separate city and mayor

In [41]:
split_list = [str(x).splitlines() for x in p[6:-2]]
split_list[:5]

[['<p>Mayor Trish Herrera Spencer<br/>', 'City of Alameda, CA</p>'],
 ['<p>Mayor Peggy McQuaid<br/>', 'City of Albany, CA</p>'],
 ['<p>Mayor Sharon Konopa<br/>', 'City of Albany, OR</p>'],
 ['<p>Mayor Kathy Sheehan<br/>', 'City of Albany, NY</p>'],
 ['<p>Mayor Tim Keller<br/>', 'City of Albuquerque, NM</p>']]

create a dataframe from our list

In [42]:
mayor_paris = pd.DataFrame(split_list)
mayor_paris.head()

,0,1
0,<p>Mayor Trish Herrera Spencer<br/>,"City of Alameda, CA</p>"
1,<p>Mayor Peggy McQuaid<br/>,"City of Albany, CA</p>"
2,<p>Mayor Sharon Konopa<br/>,"City of Albany, OR</p>"
3,<p>Mayor Kathy Sheehan<br/>,"City of Albany, NY</p>"
4,<p>Mayor Tim Keller<br/>,"City of Albuquerque, NM</p>"


rename columns

In [43]:
mayor_paris = mayor_paris.rename({0:'mayor_name', 1:'mayor_city'}, axis = 1)
mayor_paris.head()

,mayor_name,mayor_city
0,<p>Mayor Trish Herrera Spencer<br/>,"City of Alameda, CA</p>"
1,<p>Mayor Peggy McQuaid<br/>,"City of Albany, CA</p>"
2,<p>Mayor Sharon Konopa<br/>,"City of Albany, OR</p>"
3,<p>Mayor Kathy Sheehan<br/>,"City of Albany, NY</p>"
4,<p>Mayor Tim Keller<br/>,"City of Albuquerque, NM</p>"


Remove the tags from each column and adjust for 3 differently spelled columns from the GHG emissions dataset to make for a succesful merge

In [44]:
if len(mayor_paris['mayor_city'][0]) >= 12:
    mayor_paris['mayor_name'] = mayor_paris['mayor_name'].str[9:-5]
    mayor_paris['mayor_city'] = mayor_paris['mayor_city'].str[8:-4]
    
if mayor_paris.at[292, 'mayor_city'] != 'New York, NY':
    mayor_paris.at[292, 'mayor_city'] = 'New York, NY'
    
if mayor_paris.at[434, 'mayor_city'] != 'Washington D.C., DC':
    mayor_paris.at[434, 'mayor_city'] = 'Washington D.C., DC'

if mayor_paris.at[396, 'mayor_city'] != 'St. Louis, MO':
    mayor_paris.at[396, 'mayor_city'] = 'St. Louis, MO'

mayor_paris.head(10)

,mayor_name,mayor_city
0,Trish Herrera Spencer,"Alameda, CA"
1,Peggy McQuaid,"Albany, CA"
2,Sharon Konopa,"Albany, OR"
3,Kathy Sheehan,"Albany, NY"
4,Tim Keller,"Albuquerque, NM"
5,Justin Wilson,"Alexandria, VA"
6,Ray O’Connell,"Allentown, PA"
7,Jeanne Sorg,"Ambler, PA"
8,Gary Goosman,"Amesville, OH"
9,Gavin Buckley,"Annapolis, MD"


# 3. City Partisan Index

read the csv file from the study

In [45]:
city_partisan = pd.read_csv(path.join(DATA_DIR, 'city_partisan_index.csv'))
city_partisan.head()

,City,Conservatism
0,"Mesa, AZ",0.41
1,"Oklahoma City, OK",0.26
2,"Virginia Beach, VA",0.25
3,"Colorado Springs, CO",0.23
4,"Jacksonville, FL",0.18


this is all we need to do with this one right now

# 4. Merge 1-3

## Review datasets 1-3

1. GHG Emissions per capita as city_us

In [46]:
city_us.head()

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13
2,Cleveland,USA,973000,NaN,8.8,24.7 &plusmn;9.0,726,17
3,St. Louis,USA,1177000,NaN,9.6,24.1 &plusmn;8.2,448,19
4,Pittsburgh,USA,602000,NaN,6.3,24.0 &plusmn;10.4,753,21


2. Mayoral Support for Paris Climate Agreement as mayor_paris

In [47]:
mayor_paris.head()

,mayor_name,mayor_city
0,Trish Herrera Spencer,"Alameda, CA"
1,Peggy McQuaid,"Albany, CA"
2,Sharon Konopa,"Albany, OR"
3,Kathy Sheehan,"Albany, NY"
4,Tim Keller,"Albuquerque, NM"


3. City Partisan Index as city_partisan

In [48]:
city_partisan.head()

,City,Conservatism
0,"Mesa, AZ",0.41
1,"Oklahoma City, OK",0.26
2,"Virginia Beach, VA",0.25
3,"Colorado Springs, CO",0.23
4,"Jacksonville, FL",0.18


## 4.1 Combine city_us (GHG emissions per capita) and city_partisan (City Partisan Index)

Remove the state part of city_partisan

In [49]:
if len(city_partisan['City'][0]) >= 5:
    city_partisan['City'] = city_partisan['City'].str[:-4]
city_partisan.head()

,City,Conservatism
0,Mesa,0.41
1,Oklahoma City,0.26
2,Virginia Beach,0.25
3,Colorado Springs,0.23
4,Jacksonville,0.18


merge both on their respective city column, 'Urban Cluster' and 'City'

In [50]:
comb_1 = pd.merge(city_us, city_partisan, how='left', left_on = 'Urban Cluster', right_on = 'City')
comb_1.head()

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking,City,Conservatism
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12,New Orleans,-0.51
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13,Detroit,-0.73
2,Cleveland,USA,973000,NaN,8.8,24.7 &plusmn;9.0,726,17,Cleveland,-0.42
3,St. Louis,USA,1177000,NaN,9.6,24.1 &plusmn;8.2,448,19,St. Louis,-0.52
4,Pittsburgh,USA,602000,NaN,6.3,24.0 &plusmn;10.4,753,21,NaN,NaN


### Find values lost in the 1st merge

Create a list of the Urban Cluster and City column from comb_1 and a list of the city column from the city_partisan table. And check length of all.

In [51]:
comb_1_city = (comb_1['City'].dropna()).tolist()
comb_1_urban = (comb_1['Urban Cluster'].dropna()).tolist()
city_partisan_city = (city_partisan['City'].dropna()).tolist()
print(len(comb_1_city))
print(len(comb_1_urban))
print(len(city_partisan_city))

50
109
67


Find cities I lost in the merge from city_partisan. It should be 17.

In [52]:
city_partisan_lost = []
for city in city_partisan_city:
    if city not in comb_1_city:
        city_partisan_lost.append(city)

print(city_partisan_lost)
print(len(city_partisan_lost))

['Mesa', 'Virginia Beach', 'Arlington', 'Anaheim', 'Aurora', 'Anchorage', 'Fort Worth', 'Lexington', 'Riverside', 'Santa Ana', 'Long Beach', 'Honolulu', 'Newark', 'Pittrburgh', 'St. Paul', 'Washington', 'San Francisco']
17


17 as we would expect (67-50). We will take note of Pittsburgh and Washington as they are spelled differently. The ghg dataset doesn't have San Francisco so that makes sense.

In [53]:
comb_1.head()

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking,City,Conservatism
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12,New Orleans,-0.51
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13,Detroit,-0.73
2,Cleveland,USA,973000,NaN,8.8,24.7 &plusmn;9.0,726,17,Cleveland,-0.42
3,St. Louis,USA,1177000,NaN,9.6,24.1 &plusmn;8.2,448,19,St. Louis,-0.52
4,Pittsburgh,USA,602000,NaN,6.3,24.0 &plusmn;10.4,753,21,NaN,NaN


## 4.2 Put mayor support for climate change in as boolean

## Merge and create boolean column

here are all the mayors and cities that signed support for the Paris Climate accord

In [54]:
mayor_paris

,mayor_name,mayor_city
0,Trish Herrera Spencer,"Alameda, CA"
1,Peggy McQuaid,"Albany, CA"
2,Sharon Konopa,"Albany, OR"
3,Kathy Sheehan,"Albany, NY"
4,Tim Keller,"Albuquerque, NM"
...,...,...
461,Brian Sager,"Woodstock, IL"
462,Joseph M. Petty,"Worcester, MA"
463,Mike Spano,"Yonkers, NY"
464,Jamael Tito Brown,"Youngstown, OH"


Take the state off the 'mayor_city' column to set up for merge

In [55]:
if len(mayor_paris['mayor_city'][0]) >= 8:
    mayor_paris['mayor_city'] = mayor_paris['mayor_city'].str[:-4]
mayor_paris

,mayor_name,mayor_city
0,Trish Herrera Spencer,Alameda
1,Peggy McQuaid,Albany
2,Sharon Konopa,Albany
3,Kathy Sheehan,Albany
4,Tim Keller,Albuquerque
...,...,...
461,Brian Sager,Woodstock
462,Joseph M. Petty,Worcester
463,Mike Spano,Yonkers
464,Jamael Tito Brown,Youngstown


combine mayor_paris and comb_1

In [56]:
city_dataset = pd.merge(comb_1, mayor_paris, how='left', left_on = 'Urban Cluster', right_on = 'mayor_city')
city_dataset

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking,City,Conservatism,mayor_name,mayor_city
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12,New Orleans,-0.51,LaToya Cantrell,New Orleans
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13,Detroit,-0.73,Mike Duggan,Detroit
2,Cleveland,USA,973000,NaN,8.8,24.7 &plusmn;9.0,726,17,Cleveland,-0.42,Frank Jackson,Cleveland
3,St. Louis,USA,1177000,NaN,9.6,24.1 &plusmn;8.2,448,19,St. Louis,-0.52,Lyda Krewson,St. Louis
4,Pittsburgh,USA,602000,NaN,6.3,24.0 &plusmn;10.4,753,21,NaN,NaN,William Peduto,Pittsburgh
...,...,...,...,...,...,...,...,...,...,...,...,...
111,Unknown city at lat/lon 40.3,USA,253000,NaN,2.1,11.0 &plusmn;8.2,193,330,NaN,NaN,NaN,NaN
112,Bakersfield,USA,446000,NaN,2.9,11.0 &plusmn;6.5,30,331,NaN,NaN,NaN,NaN
113,Riverside-San Bernardino,USA,166000,NaN,1.4,10.9 &plusmn;8.7,49,335,NaN,NaN,NaN,NaN
114,Unknown city at lat/lon 33.5,USA,193000,NaN,1.3,9.6 &plusmn;6.9,39,425,NaN,NaN,NaN,NaN


create boolean column on whether the cities mayor signed support

In [57]:
city_dataset['supports_paris'] = city_dataset['Urban Cluster'] == city_dataset['mayor_city']
city_dataset

,Urban Cluster,Country,Population,Footprint (Mt CO2),1StdDev,Footprint/Cap (t CO2/cap),ClusterID,Global ranking,City,Conservatism,mayor_name,mayor_city,supports_paris
0,New Orleans,USA,596000,NaN,7.8,26.1 &plusmn;13.0,373,12,New Orleans,-0.51,LaToya Cantrell,New Orleans,True
1,Detroit,USA,2770000,NaN,15.4,25.5 &plusmn;5.6,16079,13,Detroit,-0.73,Mike Duggan,Detroit,True
2,Cleveland,USA,973000,NaN,8.8,24.7 &plusmn;9.0,726,17,Cleveland,-0.42,Frank Jackson,Cleveland,True
3,St. Louis,USA,1177000,NaN,9.6,24.1 &plusmn;8.2,448,19,St. Louis,-0.52,Lyda Krewson,St. Louis,True
4,Pittsburgh,USA,602000,NaN,6.3,24.0 &plusmn;10.4,753,21,NaN,NaN,William Peduto,Pittsburgh,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,Unknown city at lat/lon 40.3,USA,253000,NaN,2.1,11.0 &plusmn;8.2,193,330,NaN,NaN,NaN,NaN,False
112,Bakersfield,USA,446000,NaN,2.9,11.0 &plusmn;6.5,30,331,NaN,NaN,NaN,NaN,False
113,Riverside-San Bernardino,USA,166000,NaN,1.4,10.9 &plusmn;8.7,49,335,NaN,NaN,NaN,NaN,False
114,Unknown city at lat/lon 33.5,USA,193000,NaN,1.3,9.6 &plusmn;6.9,39,425,NaN,NaN,NaN,NaN,False


rename and reduce the number of columns

In [58]:
if 'Urban Cluster' in city_dataset.columns:
    city_dataset = city_dataset[['Urban Cluster', 'mayor_name','Footprint/Cap (t CO2/cap)', 'Conservatism', 'supports_paris']]
    city_dataset = city_dataset.rename(columns = {'Urban Cluster': 'city', 'Footprint/Cap (t CO2/cap)':'CO2_per_capita', 'Conservatism':'conservatism' })
city_dataset

,city,mayor_name,CO2_per_capita,conservatism,supports_paris
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True
2,Cleveland,Frank Jackson,24.7 &plusmn;9.0,-0.42,True
3,St. Louis,Lyda Krewson,24.1 &plusmn;8.2,-0.52,True
4,Pittsburgh,William Peduto,24.0 &plusmn;10.4,NaN,True
...,...,...,...,...,...
111,Unknown city at lat/lon 40.3,NaN,11.0 &plusmn;8.2,NaN,False
112,Bakersfield,NaN,11.0 &plusmn;6.5,NaN,False
113,Riverside-San Bernardino,NaN,10.9 &plusmn;8.7,NaN,False
114,Unknown city at lat/lon 33.5,NaN,9.6 &plusmn;6.9,NaN,False


## Correct for cities from city_partisan that didn't merge

now I want to bring back the city_partisan_lost list

In [59]:
city_partisan_lost

['Mesa',
 'Virginia Beach',
 'Arlington',
 'Anaheim',
 'Aurora',
 'Anchorage',
 'Fort Worth',
 'Lexington',
 'Riverside',
 'Santa Ana',
 'Long Beach',
 'Honolulu',
 'Newark',
 'Pittrburgh',
 'St. Paul',
 'Washington',
 'San Francisco']

In this we see Pittsburgh is spelled wrong and Washington for Washington D.C. is spelled differently when compared to the main dataset

In [60]:
print(city_partisan_lost[-2], city_partisan_lost[-4])
print(city_dataset.iloc[4,0], city_dataset.iloc[40,0])

Washington Pittrburgh
Pittsburgh Washington D.C.


This is why the two cities didn't merge!
I will create a sub datadrame with just these values from city_partisan and merge with city_datset to get their conservatism score in the city_dataset

In [61]:
sub_city_partisan = city_partisan.loc[[48, 65]]
sub_city_partisan = sub_city_partisan.rename(columns = {'City':'city', 'Conservatism': 'conservatism'})
sub_city_partisan

,city,conservatism
48,Pittrburgh,-0.49
65,Washington,-0.93


Change their spelling

In [62]:
sub_city_partisan['city'].replace({'Pittrburgh': 'Pittsburgh', 'Washington': 'Washington D.C.'}, inplace = True)
sub_city_partisan

,city,conservatism
48,Pittsburgh,-0.49
65,Washington D.C.,-0.93


merge

In [63]:
city_dataset = pd.merge(city_dataset, sub_city_partisan, how='left', left_on = 'city', right_on = 'city')
city_dataset

,city,mayor_name,CO2_per_capita,conservatism_x,supports_paris,conservatism_y
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True,NaN
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True,NaN
2,Cleveland,Frank Jackson,24.7 &plusmn;9.0,-0.42,True,NaN
3,St. Louis,Lyda Krewson,24.1 &plusmn;8.2,-0.52,True,NaN
4,Pittsburgh,William Peduto,24.0 &plusmn;10.4,NaN,True,-0.49
...,...,...,...,...,...,...
111,Unknown city at lat/lon 40.3,NaN,11.0 &plusmn;8.2,NaN,False,NaN
112,Bakersfield,NaN,11.0 &plusmn;6.5,NaN,False,NaN
113,Riverside-San Bernardino,NaN,10.9 &plusmn;8.7,NaN,False,NaN
114,Unknown city at lat/lon 33.5,NaN,9.6 &plusmn;6.9,NaN,False,NaN


In [64]:
city_dataset

,city,mayor_name,CO2_per_capita,conservatism_x,supports_paris,conservatism_y
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True,NaN
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True,NaN
2,Cleveland,Frank Jackson,24.7 &plusmn;9.0,-0.42,True,NaN
3,St. Louis,Lyda Krewson,24.1 &plusmn;8.2,-0.52,True,NaN
4,Pittsburgh,William Peduto,24.0 &plusmn;10.4,NaN,True,-0.49
...,...,...,...,...,...,...
111,Unknown city at lat/lon 40.3,NaN,11.0 &plusmn;8.2,NaN,False,NaN
112,Bakersfield,NaN,11.0 &plusmn;6.5,NaN,False,NaN
113,Riverside-San Bernardino,NaN,10.9 &plusmn;8.7,NaN,False,NaN
114,Unknown city at lat/lon 33.5,NaN,9.6 &plusmn;6.9,NaN,False,NaN


clean the merge

In [65]:
if 'conservatism_y' in city_dataset.columns:
    city_dataset.conservatism_x.fillna(city_dataset.conservatism_y, inplace = True)
    city_dataset.drop(['conservatism_y'], axis = 1, inplace = True)
city_dataset = city_dataset.rename(columns={'conservatism_x': 'conservatism'})
city_dataset.head()

,city,mayor_name,CO2_per_capita,conservatism,supports_paris
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True
2,Cleveland,Frank Jackson,24.7 &plusmn;9.0,-0.42,True
3,St. Louis,Lyda Krewson,24.1 &plusmn;8.2,-0.52,True
4,Pittsburgh,William Peduto,24.0 &plusmn;10.4,-0.49,True


## Final after Merge

In [66]:
city_dataset

,city,mayor_name,CO2_per_capita,conservatism,supports_paris
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True
2,Cleveland,Frank Jackson,24.7 &plusmn;9.0,-0.42,True
3,St. Louis,Lyda Krewson,24.1 &plusmn;8.2,-0.52,True
4,Pittsburgh,William Peduto,24.0 &plusmn;10.4,-0.49,True
...,...,...,...,...,...
111,Unknown city at lat/lon 40.3,NaN,11.0 &plusmn;8.2,NaN,False
112,Bakersfield,NaN,11.0 &plusmn;6.5,NaN,False
113,Riverside-San Bernardino,NaN,10.9 &plusmn;8.7,NaN,False
114,Unknown city at lat/lon 33.5,NaN,9.6 &plusmn;6.9,NaN,False


# 5. Clean merged dataset and add Mayor Political Party

eliminate all rows that don't have a conservatism scale.

In [67]:
city_dataset = city_dataset.loc[city_dataset['conservatism'].notna()]
city_dataset.head()

,city,mayor_name,CO2_per_capita,conservatism,supports_paris
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True
2,Cleveland,Frank Jackson,24.7 &plusmn;9.0,-0.42,True
3,St. Louis,Lyda Krewson,24.1 &plusmn;8.2,-0.52,True
4,Pittsburgh,William Peduto,24.0 &plusmn;10.4,-0.49,True


split the 'CO2_per_capita' column into 2 with 'plus_minus' as the other column

In [68]:
pd.options.mode.chained_assignment = None
city_dataset['CO2_per_cap'] = city_dataset['CO2_per_capita'].str.split(' &plusmn;').str[0]
city_dataset['plus_minus'] = city_dataset['CO2_per_capita'].str.split(' &plusmn;').str[1]
city_dataset

,city,mayor_name,CO2_per_capita,conservatism,supports_paris,CO2_per_cap,plus_minus
0,New Orleans,LaToya Cantrell,26.1 &plusmn;13.0,-0.51,True,26.1,13.0
1,Detroit,Mike Duggan,25.5 &plusmn;5.6,-0.73,True,25.5,5.6
2,Cleveland,Frank Jackson,24.7 &plusmn;9.0,-0.42,True,24.7,9.0
3,St. Louis,Lyda Krewson,24.1 &plusmn;8.2,-0.52,True,24.1,8.2
4,Pittsburgh,William Peduto,24.0 &plusmn;10.4,-0.49,True,24.0,10.4
5,Toledo,Wade Kapszukiewicz,22.8 &plusmn;13.5,-0.17,True,22.8,13.5
6,Kansas City,Quinton Lucas,22.6 &plusmn;7.4,-0.38,True,22.6,7.4
8,Cincinnati,John Cranley,22.6 &plusmn;8.9,-0.22,True,22.6,8.9
9,Tulsa,NaN,22.6 &plusmn;10.3,0.07,False,22.6,10.3
12,Minneapolis,Jacob Frey,21.8 &plusmn;6.4,-0.77,True,21.8,6.4


In [69]:
pd.options.mode.chained_assignment = 'warn'
city_dataset = city_dataset[['city', 'mayor_name', 'CO2_per_cap', 'plus_minus', 'conservatism', 'supports_paris']]
city_dataset

,city,mayor_name,CO2_per_cap,plus_minus,conservatism,supports_paris
0,New Orleans,LaToya Cantrell,26.1,13.0,-0.51,True
1,Detroit,Mike Duggan,25.5,5.6,-0.73,True
2,Cleveland,Frank Jackson,24.7,9.0,-0.42,True
3,St. Louis,Lyda Krewson,24.1,8.2,-0.52,True
4,Pittsburgh,William Peduto,24.0,10.4,-0.49,True
5,Toledo,Wade Kapszukiewicz,22.8,13.5,-0.17,True
6,Kansas City,Quinton Lucas,22.6,7.4,-0.38,True
8,Cincinnati,John Cranley,22.6,8.9,-0.22,True
9,Tulsa,NaN,22.6,10.3,0.07,False
12,Minneapolis,Jacob Frey,21.8,6.4,-0.77,True


create last column for mayor political party

In [70]:
city_dataset['mayor_political_party'] = np.nan
city_dataset.head()

,city,mayor_name,CO2_per_cap,plus_minus,conservatism,supports_paris,mayor_political_party
0,New Orleans,LaToya Cantrell,26.1,13.0,-0.51,True,NaN
1,Detroit,Mike Duggan,25.5,5.6,-0.73,True,NaN
2,Cleveland,Frank Jackson,24.7,9.0,-0.42,True,NaN
3,St. Louis,Lyda Krewson,24.1,8.2,-0.52,True,NaN
4,Pittsburgh,William Peduto,24.0,10.4,-0.49,True,NaN


Remove duplicate cities

In [71]:
city_dataset['city'].duplicated(keep = False)

0      False
1      False
2      False
3      False
4      False
5      False
6      False
8      False
9      False
12     False
14     False
16     False
19     False
22     False
23     False
24     False
26     False
29     False
30     False
31     False
33     False
34     False
35     False
36     False
39     False
40     False
43     False
50     False
52     False
53     False
56     False
59     False
61     False
62     False
63     False
66      True
67      True
72     False
74     False
82     False
84     False
90     False
91     False
92     False
95     False
96     False
97     False
98     False
99     False
100    False
103    False
106    False
107    False
Name: city, dtype: bool

Portland only one what was a duplicate

In [72]:
city_dataset.loc[[66, 67]]

,city,mayor_name,CO2_per_cap,plus_minus,conservatism,supports_paris,mayor_political_party
66,Portland,Ethan Strimling,16.7,8.0,-0.59,True,NaN
67,Portland,Ted Wheeler,16.7,8.0,-0.59,True,NaN


Only keep the Portland from Oregon.

In [73]:
if city_dataset['mayor_name'].loc[66] == 'Ethan Strimling':
    city_dataset.drop([66], inplace = True)
city_dataset

,city,mayor_name,CO2_per_cap,plus_minus,conservatism,supports_paris,mayor_political_party
0,New Orleans,LaToya Cantrell,26.1,13.0,-0.51,True,NaN
1,Detroit,Mike Duggan,25.5,5.6,-0.73,True,NaN
2,Cleveland,Frank Jackson,24.7,9.0,-0.42,True,NaN
3,St. Louis,Lyda Krewson,24.1,8.2,-0.52,True,NaN
4,Pittsburgh,William Peduto,24.0,10.4,-0.49,True,NaN
5,Toledo,Wade Kapszukiewicz,22.8,13.5,-0.17,True,NaN
6,Kansas City,Quinton Lucas,22.6,7.4,-0.38,True,NaN
8,Cincinnati,John Cranley,22.6,8.9,-0.22,True,NaN
9,Tulsa,NaN,22.6,10.3,0.07,False,NaN
12,Minneapolis,Jacob Frey,21.8,6.4,-0.77,True,NaN


Address missing mayor names

In [74]:
city_dataset[['city', 'mayor_name']].loc[city_dataset['mayor_name'].isna()]

,city,mayor_name
9,Tulsa,NaN
16,Wichita,NaN
22,Oklahoma City,NaN
30,Omaha,NaN
50,Colorado Springs,NaN
63,Jacksonville,NaN
72,Corpus Christi,NaN
98,El Paso,NaN
106,Fresno,NaN
107,Las Vegas,NaN


Create a dict for mayor names and a dataframe

In [75]:
mayor_names = {'Tulsa': 'Dewey F. Bartlett Jr.', 'Wichita': 'Jeff Longwell', 'Oklahoma City': 'Mick Cornett', 'Omaha': 'Jean Stothert',
              'Colorado Springs': 'John Suthers', 'Jacksonville': 'Lenny Curry', 'Corpus Christi': 'Joe McComb', 'El Paso': 'Dee Margo',
              'Fresno': 'Lee Brand', 'Las Vegas': 'Carolyn Goodman'}

key_list = list(mayor_names.keys())
val_list = list(mayor_names.values())
mayor_df = DataFrame(key_list)  
mayor_df['mayor_name'] = val_list
mayor_df.rename(columns = {0: 'city'}, inplace = True)
mayor_df

,city,mayor_name
0,Tulsa,Dewey F. Bartlett Jr.
1,Wichita,Jeff Longwell
2,Oklahoma City,Mick Cornett
3,Omaha,Jean Stothert
4,Colorado Springs,John Suthers
5,Jacksonville,Lenny Curry
6,Corpus Christi,Joe McComb
7,El Paso,Dee Margo
8,Fresno,Lee Brand
9,Las Vegas,Carolyn Goodman


merge missing mayors table this with the main dataset

In [76]:
city_dataset_merge = pd.merge(city_dataset, mayor_df, how='left', on = 'city')
city_dataset_merge

city_dataset_merge['mayor_name'] = city_dataset_merge['mayor_name_x'].fillna(city_dataset_merge['mayor_name_y'])
city_dataset_merge = city_dataset_merge.drop(['mayor_name_x', 'mayor_name_y'], axis = 1)
city_dataset = city_dataset_merge[['city', 'mayor_name','CO2_per_cap', 
                                   'plus_minus', 'conservatism', 'supports_paris',  'mayor_political_party']]
city_dataset

,city,mayor_name,CO2_per_cap,plus_minus,conservatism,supports_paris,mayor_political_party
0,New Orleans,LaToya Cantrell,26.1,13.0,-0.51,True,NaN
1,Detroit,Mike Duggan,25.5,5.6,-0.73,True,NaN
2,Cleveland,Frank Jackson,24.7,9.0,-0.42,True,NaN
3,St. Louis,Lyda Krewson,24.1,8.2,-0.52,True,NaN
4,Pittsburgh,William Peduto,24.0,10.4,-0.49,True,NaN
5,Toledo,Wade Kapszukiewicz,22.8,13.5,-0.17,True,NaN
6,Kansas City,Quinton Lucas,22.6,7.4,-0.38,True,NaN
7,Cincinnati,John Cranley,22.6,8.9,-0.22,True,NaN
8,Tulsa,Dewey F. Bartlett Jr.,22.6,10.3,0.07,False,NaN
9,Minneapolis,Jacob Frey,21.8,6.4,-0.77,True,NaN


After searching for possible sites to webscrape on, it will be easiest to fill in this column manually

export to a csv file with nan values to be filled in

In [77]:
city_dataset1 = city_dataset.sort_values('mayor_name')
city_dataset1.to_csv(r'/Users/joeskvarna/Desktop/Thesis/Data/city_dataset_fillin.csv', index = False)

Final dataset after filling in last column

In [78]:
city_dataset = pd.read_csv(path.join(DATA_DIR, 'city_dataset_reimport.csv'))
city_dataset

,city,mayor_name,CO2_per_cap,plus_minus,conservatism,supports_paris,mayor_political_party
0,Columbus,Andrew Ginther,19.8,8.5,-0.26,True,D
1,Baltimore,Bernard Young,20.2,9.1,-0.66,True,D
2,New York,Bill de Blasio,17.1,5.5,-0.66,True,D
3,Buffalo,Byron Brown,19.8,10.0,-0.66,True,D
4,Las Vegas,Carolyn Goodman,12.3,4.8,-0.03,False,I
5,Sacramento,Darrell Steinberg,14.2,4.9,-0.33,True,D
6,El Paso,Dee Margo,14.6,8.5,-0.07,False,R
7,Tulsa,Dewey F. Bartlett Jr.,22.6,10.3,0.07,False,R
8,Los Angeles,Eric Garcetti,14.6,3.2,-0.49,True,D
9,Dallas,Eric Johnson,16.5,3.4,-0.23,True,D


Save to path

In [79]:
city_dataset.to_csv('/Users/joeskvarna/Desktop/Thesis/Data/city_dataset.csv', index = False)